In [1]:
import pandas as pd

In [16]:
df_imu = pd.read_csv(
    "D:/thesis_code/data/EuRoC/MOO11_short_3_backandforth/mav0/imu0/data.csv",
    header=0,
    names=(
        'timestamp', 
        'accel_x', 'accel_y', 'accel_z', 
        'gyro_x', 'gyro_y', 'gyro_z'
    )
)

df_imu['timestamp'] = pd.to_datetime(df_imu['timestamp'], unit='ns')

df_imu.head()

,timestamp,accel_x,accel_y,accel_z,gyro_x,gyro_y,gyro_z
0,1970-01-01 02:29:48.972303690,-0.003026,-0.003380,0.001754,0.051727,-9.677321,1.002005
1,1970-01-01 02:29:48.973303690,0.002137,0.000993,0.000764,0.051727,-9.667402,0.973449
2,1970-01-01 02:29:48.974298990,-0.001732,0.012570,-0.005921,0.037855,-9.643586,0.988266
3,1970-01-01 02:29:48.975294190,0.000958,-0.019189,0.003014,0.037855,-9.610852,0.988295
4,1970-01-01 02:29:48.976284590,0.001921,0.016278,0.001371,0.041819,-9.595976,0.993228


In [17]:
df_image_cam0 = pd.read_csv(
    "D:/thesis_code/data/EuRoC/MOO11_short_3_backandforth/mav0/cam0/data.csv",
    header=0,
    names=(
        'timestamp', 
        'filename'
    )
)

df_image_cam0['timestamp'] = pd.to_datetime(df_image_cam0['timestamp'], unit='ns')
df_image_cam0 = df_image_cam0.sort_values(by='timestamp')
df_image_cam0['filename'] = df_image_cam0['filename'].str.replace(".png", "", regex=False)

df_image_cam0.head()

,timestamp,filename
0,1970-01-01 02:29:48.980098554,8988981394090
1,1970-01-01 02:29:49.013387754,8989014683290
2,1970-01-01 02:29:49.046676254,8989047971790
3,1970-01-01 02:29:49.079967654,8989081263190
4,1970-01-01 02:29:49.113255254,8989114550790


In [27]:
def average_last_n_imu_data(imu_df, image_df, N=10):
    averaged_imu_data = []

    for _, image_row in image_df.iterrows():
        image_timestamp = image_row["timestamp"]
        filename = image_row["filename"]
        
        # Last N IMU rows before current image timestamp
        # len = 13536
        mask = imu_df["timestamp"] <= image_timestamp 
        imu_before = imu_df[mask].tail(N)

        averaged_imu_data.append({
            "timestamp": image_timestamp,
            "filename": filename,
            
            "avg_accel_x": imu_before["accel_x"].mean(),
            "avg_accel_y": imu_before["accel_y"].mean(),
            "avg_accel_z": imu_before["accel_z"].mean(),
            
            "avg_gyro_x": imu_before["gyro_x"].mean(),
            "avg_gyro_y": imu_before["gyro_y"].mean(),
            "avg_gyro_z": imu_before["gyro_z"].mean()
        })

    return pd.DataFrame(averaged_imu_data)

In [28]:
df = average_last_n_imu_data(df_imu, df_image_cam0, N=50)
df.head()

,timestamp,filename,avg_accel_x,avg_accel_y,avg_accel_z,avg_gyro_x,avg_gyro_y,avg_gyro_z
0,1970-01-01 02:29:48.980098554,8988981394090,-0.000045,0.001914,-0.000164,0.046401,-9.612966,0.992221
1,1970-01-01 02:29:49.013387754,8989014683290,0.004635,0.000052,-0.001792,0.060267,-9.656143,1.049923
2,1970-01-01 02:29:49.046676254,8989047971790,0.001968,-0.000109,-0.001991,0.063121,-9.651044,1.076436
3,1970-01-01 02:29:49.079967654,8989081263190,-0.001261,-0.000855,0.002117,0.063359,-9.659852,1.059112
4,1970-01-01 02:29:49.113255254,8989114550790,-0.004574,-0.001154,0.001606,0.039778,-9.659775,1.063402


In [35]:
df = pd.DataFrame([[1, 2], [3, 4]], columns=['A', 'B'])

store = pd.HDFStore("store.h5", 'w')  

In [36]:
store.put('data', df)  

In [37]:
store.get('data')

,A,B
0,1,2
1,3,4


In [38]:
store.close() 